<blockquote><b><i><h1 style="font-family: Arial ; text-align:center;">Studying and explaining the mushroom dataset</h1></i></b></blockquote>

<img src="http://cdn6.bigcommerce.com/s-tu5jcd/product_images/uploaded_images/mushroom-parts-labeled-small.jpg?t=1456508398" style="width:350px;height:350px;">

In [ ]:
import subprocess
import sys
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip","install",package])
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.svm import SVC
from IPython.display import Image
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import scipy.stats as ss
import math
from sklearn.feature_selection import SelectKBest, chi2
import plotly.figure_factory as ff
from sklearn.feature_selection import SequentialFeatureSelector
import plotly.express as px
import plotly.graph_objs as go
from plotly import tools
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

class data_transf:
    def __init__(self,data,selection=0,split=True):
        self.data = data
        self.split = split
    def bina(self,x):
        if x=='e':
            return 0
        else:
            return 1
    
    def modify(self):

        if self.split:
            split = StratifiedShuffleSplit(n_splits=2, test_size=0.2, random_state=10)
            for train_index, test_index in split.split(self.data, self.data[["class","odor",'gill-attachment','stalk-surface-below-ring','stalk-surface-below-ring','gill-color']]):
                train_set = self.data.loc[train_index]
                test_set = self.data.loc[test_index]
            

            binary_test=test_set['class'].apply(self.bina)
            test_set=pd.concat([test_set.drop('class',axis=1),binary_test],axis=1)
            test_set=pd.get_dummies(test_set)
             
            binary_train=train_set['class'].apply(self.bina)
            train_set=pd.concat([train_set.drop('class',axis=1),binary_train],axis=1)
            train_set=pd.get_dummies(train_set)
            
            a=[]
            for col in train_set.columns:
                for x in train_set[col].value_counts():
                    if col not in test_set.columns:
                        train_set.drop(col,axis=1,inplace=True)
                        break
                    elif x<5:
                        train_set.drop(col,axis=1,inplace=True)
                        test_set.drop(col,axis=1,inplace=True)
                        

            return train_set.drop('class',axis=1),test_set.drop('class',axis=1),train_set['class'],test_set['class']

        return self.data
def bina(x):
    if x=='e':
        return 0
    else:
        return 1

mush_data=pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')
mush_data

**This is a classical dataset that everyone that decides to study machine learning knows, the mushroom dataset.** <br>
**But for those who don't: The purpose of the study is to produce a classification algorithm capable of differentiating between poisonous and edible mushrooms.** <br>
**This is the first time I try to tackle this project and I decided to proceed in an ordered manner following each classical step that needs to be made for a machine learning project.**<br>
**Sadly this turned out to be useless in this specific case:**

In [ ]:
obj=data_transf(mush_data)
X_train,X_test,y_train,y_test=obj.modify()
clf = DecisionTreeClassifier(random_state=0,max_depth=5)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print('Accuracy_score={A}'.format(A=accuracy_score(y_test, y_pred)))
print('f1_score={f1}'.format(f1=f1_score(y_test, y_pred)))

**As we can see if we randomly divide our dataset in a test and training subsets, do the needed transformations and run a DecisionTreeClassifier both the accuracy and the f1 score of the model on the test are 1.** <br> 
**This doesn't mean that the model is overfitting (since I'm testing it on the test set), but that this is an extremely easy classification problem for the algorithms to solve.**<br>
**I tried to explore what is happening later on in the notebook.**

<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">

<p style="padding: 10px;
              color:white;">

**WARNING:** If you are interested in the steps that It would be normal to take during a classification problem go on, if you are only interested in some insights on why this is such un easy problem you can go directly to the explanation paragraph   
</p>
</div>

# Data Exploration 

**The First thing we want to do is understand our data. Let's start by looking at our features' types** 

In [ ]:
colorscale = [[0, '#4d004c'],[.5, '#f2e5ff'],[1, '#ffffff']]
fig =  ff.create_table(pd.DataFrame({'col':mush_data.columns,'types':mush_data.dtypes}).reset_index(drop=True),height_constant=16, colorscale=colorscale,index=True)
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].font.size = 13
fig.layout.width=500
fig

**Ok, now that we know we are dealing only with categorical variables we can estabilish what to do next. There are 22 features in total and for each of them we need to see the proportions of the various entries inside. The reason for doing this is that there might be some features composed mostly or even totally by just one variable making them useless, or/and we might have some features with one variable present in only a few instances and that might force me to stratify the splitting in train/test to be sure to have the same variables in both samples.** <br>
**To do this we can use a visualization.**

In [ ]:
specs = [[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=5, cols=5, specs=specs)
a=1
b=1
xx=-0.172
yy=1.267
l=[]
for col in mush_data.columns:
    fig.add_trace(go.Pie(labels=[n for n in mush_data[col].value_counts().index], values=[v for v in mush_data[col].value_counts()*100/sum(mush_data[col].value_counts())], name=col),
              a, b)
    l.append(dict(text=col, x=xx+(0.225*b), y=yy-(0.222*a), font_size=10, showarrow=False))
    a+=1
    if a>5:
        a=1
        b+=1



fig.update(layout_title_text='Features',
           layout_showlegend=False)
  
fig.update_layout(
    title_font_family="Arial",
    title_font_size=25,
    annotations=l)

fig.update_traces(hole=.4, hoverinfo="label+percent+name",textinfo='none')

fig.show()

**As we can see some features have a variable with few istances and there's even one that has only one type of entry veil-type. This one needs to be dropped immediatly. As for the others, I dealt with the problem in a class I put in class in the first chunk of code to produce one-hot-encoded train and test subsets.**

In [ ]:
mush_data=mush_data.drop('veil-type',axis=1)
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

l1=[]
l2=[]
for col in mush_data:
    for col2 in mush_data:
        l1.append(round(cramers_v(mush_data[col],mush_data[col2])*100)/100)
    l2.append(l1)
    l1=[]
df = pd.DataFrame(np.array(l2),columns=[mush_data.columns])
df.index=df.columns
plt.figure(figsize=(13,13)) 
mask = np.triu(np.ones_like(df, dtype=bool))
sns.heatmap(df,mask=mask, annot=True)

**Here I made a correlation Matrix. To do this I defined a personalized formula Cramér's V, this formula measures association between two categorical variables, and I can use it for this dataset since all of my variables are like that. With thi piece of code I confronted all of the features 1 vs 1. It's interesting to see some really high correlations here like class and odor. The problem is that we cannot exclude some multicollinearity between our features seeing this plot. This added with the fact that some variables appear only few times in some columns we need to do some feature selection.**

# Feature Selection

In [ ]:
np.random.seed(100)


X_new = SelectKBest(chi2, k='all')
X_new.fit(X_train, y_train)

fs = SelectKBest(score_func=mutual_info_classif, k='all')
fs.fit(X_train, y_train)


coef = pd.DataFrame({'col':list(X_train.columns),'coef':[v for v in X_new.scores_]}).sort_values(by='coef')
coef['cumul']=[i for i in range(0,115)]




fig = make_subplots(rows=3, cols=1, subplot_titles=("Chi2","", "Mutual_info"))

fig.add_trace(
    go.Bar(x=coef['col'], y=coef['coef'], hovertext=[('n:'+ str(n)) for n in coef['cumul']]),
    row=1, col=1
)

coef = pd.DataFrame({'col':list(X_train.columns),'coef':[v for v in fs.scores_]}).sort_values(by='coef')
coef['cumul']=[i for i in range(0,115)]

fig.add_trace(
    go.Bar(x=coef['col'], y=coef['coef'], hovertext=[('n:'+ str(n)) for n in coef['cumul']]),
    row=3, col=1
)



fig.update_layout(height=800,width=600, title_text="Selection",showlegend=False)

fig.show()

**Here I used two of the methods sklearn offers us to select variables. Chi2 and Mutual_info selections. The problem now is 'which one should we choose?'. I decided to test how the algorithm would work with one of them,none or both.**

In [ ]:
np.random.seed(1111)
X_new = SelectKBest(chi2, k=94,)
X_new.fit(X_train, y_train)

fs = SelectKBest(score_func=mutual_info_classif, k=94)
fs.fit(X_train, y_train)

headerColor = 'grey'
colors = []
i=0
for x in range(0,114):
    if list(X_new.get_support())[x] or list(fs.get_support())[x]:
        colors.append('lightgrey')
    else:
        colors.append('darksalmon')
        i-=1
    i+=1
Results=[['discarded','kept'][list(X_new.get_support())[x] or list(fs.get_support())[x]] for x in range(0,114)]


fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Columns</b>','<b>Chi2</b>','<b>Mutual-info</b>','<b>Result</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
      values=[
        list(X_train.columns),
        list(X_new.get_support()),
        list(fs.get_support()),
        Results],
    line_color='darkslategray',
    fill_color = [colors],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
        ))
])


fig.show()

**To explain what I meant when I said both I made this plot. When both methods discard a column we delete it, otherwise we keep it.**

In [ ]:

np.random.seed(3)


l=[X_train,X_train.iloc[:,X_new.get_support(indices=True)],X_train.iloc[:,fs.get_support(indices=True)],X_train.iloc[:,list(set(X_new.get_support(indices=True))|set(fs.get_support(indices=True)))]]
l2=['chi','mutual','mix']

scorer={
    'Normal':[],
    'Chi':[],
    'mutual':[],
    'mix':[]
}

x=-1
for p in scorer:
    x+=1
    for y in range(2,15):
        a=DecisionTreeClassifier(max_depth=y)
        a.fit(X_train,y_train)
        y_pred = a.predict(X_train)
        scorer[p].append(accuracy_score(y_train, y_pred))

pt=pd.DataFrame(
    scorer
)
pt.reset_index(inplace=True)



scorer2={
    'Normal':[],
    'Chi':[],
    'mutual':[],
    'mix':[]
}

x=-1
for p in scorer2:
    x+=1
    for y in range(2,15):
        a=DecisionTreeClassifier(max_leaf_nodes=y)
        a.fit(l[x],y_train)
        y_pred = a.predict(l[x])
        scorer2[p].append(accuracy_score(y_train, y_pred))
        
        

pt2=pd.DataFrame(
    scorer2
)
pt2.reset_index(inplace=True)





fig = make_subplots(rows=1, cols=2,subplot_titles=("Increasing Max_depth", "Increasing Max_leaf_nodes"))

fig.add_trace(go.Scatter(x=pt['index'], y=pt['Normal'],
                    mode='lines+markers',
                    name='Max_Depth_Normal'),row=1,col=1)
fig.add_trace(go.Scatter(x=pt['index'], y=pt['Chi'],
                    mode='lines+markers',
                    name='Max_Depth_Chi'),row=1,col=1)
fig.add_trace(go.Scatter(x=pt['index'], y=pt['mutual'],
                    mode='lines+markers',
                    name='Max_Depth_mutual'),row=1,col=1)
fig.add_trace(go.Scatter(x=pt['index'], y=pt['mix'],
                    mode='lines+markers', name='Max_Depth_mix'),row=1,col=1)

fig.add_trace(go.Scatter(x=pt2['index'], y=pt2['Normal'],
                    mode='lines+markers',
                    name='Max_leaf_nodes_Normal'),row=1,col=2)
fig.add_trace(go.Scatter(x=pt2['index'], y=pt2['Chi'],
                    mode='lines+markers',
                    name='Max_leaf_nodes_Chi'),row=1,col=2)
fig.add_trace(go.Scatter(x=pt2['index'], y=pt2['mutual'],
                    mode='lines+markers',
                    name='Max_leaf_nodes_mutual'),row=1,col=2)
fig.add_trace(go.Scatter(x=pt2['index'], y=pt2['mix'],
                    mode='lines+markers', name='Max_leaf_nodes_mix'),row=1,col=2)






fig.update_xaxes(title_text="Max_depth", row=1, col=1)
fig.update_xaxes(title_text="Max_leaf_nodes", row=1, col=2)





fig.show()

**I made this plot to see the accuracy of a decision tree trained and tested on the train set with increasing max_leaf_nodes and max_depth. I wanted to see which one of the methods would perform the best out of all of them. All of them, aside from mutual-info, were on the same level. But one thing about these plot intrigued me, both of them overfit too quickly. This is why I made the following plot.**

In [ ]:

fig = make_subplots(rows=1, cols=2,subplot_titles=("Decision Tree",'Support-vector machine'))

scorer3={
    'Max_depth':[],
    'Accuracy':[]
}

x=-1

for y in range(2,20):
    a=DecisionTreeClassifier(max_depth=y)
    a.fit(X_train,y_train)
    y_pred = a.predict(X_test)
    scorer3['Accuracy'].append(f1_score(y_test, y_pred))
    scorer3['Max_depth'].append(y) 
        

pt3=pd.DataFrame(
    scorer3
)



scorer4={
    'gamma':[],
    'f1_score':[]
}


x=-1
d=np.arange(0.01,0.1,0.01)
for y in d:
    clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=y, C=0.001))
    ])
    clf.fit(X_train, y_train)
    clf_pred = clf.predict(X_train)
    scorer4['f1_score'].append(f1_score(y_test, y_pred))
    scorer4['gamma'].append(y) 
        

pt4=pd.DataFrame(
    scorer4
)




fig.add_trace(go.Scatter(x=pt3['Max_depth'], y=pt3['Accuracy'],name='Increasing Max_depth',
                    mode='lines+markers'),row=1,col=1)
fig.update_yaxes(title_text="f1_score", row=1, col=1)
fig.update_xaxes(title_text="Max_depth",row=1, col=1)


fig.add_trace(go.Scatter(x=pt4['gamma'], y=pt4['f1_score'],name='Increasing gamma',
                    mode='lines+markers'),row=1,col=2)
fig.update_yaxes(title_text="f1_score", row=1, col=2)
fig.update_xaxes(title_text="Increasing gamma",row=1, col=2)

**Here we have the accuracy of a decision tree and of a svm trained on the train set when used on the test set. This isn't something that would be done during a machine learning project normally but it allowed me to see what i explained in the introduction. Both these simple models are able to do this classification perfectly and easily.**

# Explanation

**There can only be two reasons for such a thing:** <br>
**1. The dataset isn't good(I doubt this is the case)** <br>
**2. There are well defined multi-dimensional clusters that allows our algorithm to divide poisonous and edible mushrooms easily**

In [ ]:
install("ivis[cpu]")
from ivis import Ivis
np.random.seed(3)
mush_y=mush_data['class'].apply(bina)
mush_X=pd.get_dummies(mush_data.drop('class',axis=1))
model = Ivis(embedding_dims=2,k=18,model='maaten',n_epochs_without_progress=18)

embeddings = model.fit_transform(mush_X.to_numpy())

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi=150)
sc = plt.scatter(x=embeddings[:, 0], y=embeddings[:, 1],c=mush_y, s=3)
plt.xlabel('ivis 1')
plt.ylabel('ivis 2')
plt.title('data')
ax.grid(True)
legend1 = ax.legend(*sc.legend_elements(),loc="lower left", title="Classes")

**Here I used Ivis a dimensionality reduction specialized in finding clusters.**<br> 
**As we can see we have really well defined groups for the two classes of mushrooms (0=edible,1=poisonous)** <br>
**This supports the second hypothesis in my previous comment**

In [ ]:
clf = DecisionTreeClassifier(random_state=0,max_depth=15)
clf.fit(X_train,y_train)
target_names=list(mush_data['class'].value_counts().index)[::-1]
export_graphviz(clf, out_file='tree_limited.dot', feature_names = X_train.columns,
                class_names = target_names,
                rounded = True, proportion = False, precision = 2, filled = True)
!dot -Tpng tree_limited.dot -o tree_limited.png -Gdpi=600

Image(filename = 'tree_limited.png')

**This is the last part of the notebook. Here we have a representation of a decision tree trained on this dataset.** <br>
**As we can see, by the time we reach depth=4 the gini impurity becomes 0 for all the leaves, this further confirms what we've seen earlier, that the decision tree is able to divide both classes perfectly, easily and quickly.**

**Hope you enjoyed my project, thanks for the attention.**